In [13]:
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()

raw_image_dataset = tf.data.TFRecordDataset('../images.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  example = tf.io.parse_single_example(example_proto, image_feature_description)
  label, image = example["label"], example["image_raw"]
  label = tf.one_hot(label, depth=10)
  # print(tf.io.parse_single_example(example_proto, image_feature_description))
  # image_raw = image_features['image_raw']
  image = tf.io.decode_raw(image, tf.uint8)
  # image = tf.reshape(image, [640, 240, 3])
  image = image.astype("float32") / 255
  
  return image, label
  
  # return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


<MapDataset shapes: ((None,), (10,)), types: (tf.float32, tf.float32)>

In [ ]:
for image_features in parsed_image_dataset:
  image = image_features[0]
  label = image_features[1]
  print(label)
  print(image.shape)
  break

In [14]:
dataset = parsed_image_dataset.shuffle(100, reshuffle_each_iteration=True).batch(64)

In [ ]:
for batch in dataset:
  print(batch)
  break

In [15]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
layer1 = layers.Dense(512, activation='relu', input_shape=(460800,))
model.add(layer1)

layer2 = layers.Dense(10, activation='softmax')
model.add(layer2)

model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               235930112 
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 235,935,242
Trainable params: 235,935,242
Non-trainable params: 0
_________________________________________________________________


In [16]:

myadam = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'
)

model.compile(optimizer=myadam,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# 降低学习率
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',verbose=1, factor=0.2,patience=1, min_lr=0.0001)

# 提前终止
es=tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1,mode='min',patience=2)
 
model.fit(dataset, epochs=10, callbacks=[reduce_lr ,es])

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
    148/Unknown - 295s 2s/step - loss: 100.6933 - accuracy: 0.1243

KeyboardInterrupt: 